In [1]:
from utils import load_dataset
import matplotlib.pyplot as plt

import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim

import torch.optim as optim

from torchvision import models, transforms
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split


In [2]:
SEED = 42
torch.manual_seed(SEED)
TRAIN_RATIO = 0.8
N_CLASSES = 2

In [3]:
train_images, train_labels = load_dataset(test=False, color=True)
test_images, test_labels = load_dataset(test=True, color=True)

In [4]:
B, W, H = train_images.shape
train_images = train_images.reshape((B, 1, W, H))
train_images = np.repeat(train_images, 3, axis=1)
train_images.shape

(800, 3, 224, 224)

In [5]:
B, W, H = test_images.shape
test_images = test_images.reshape((B, 1, W, H))
test_images = np.repeat(test_images, 3, axis=1)

test_images.shape

(200, 3, 224, 224)

In [6]:

# Charger un modèle pré-entraîné (ResNet18 dans cet exemple)
model = models.resnet18(pretrained=True)

# Geler les paramètres (poids) de toutes les couches du modèle
for param in model.parameters():
    param.requires_grad = False

# Remplacer la dernière couche de classification
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2) 

# Afficher le résumé du modèle (nécessite torchsummary)


c:\Users\gouhe\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\gouhe\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
model.fc.out_features

2

In [8]:


# Charger un modèle pré-entraîné (ResNet18)
model = models.resnet18(pretrained=True)

# Geler les paramètres (poids) de toutes les couches du modèle
for param in model.parameters():
    param.requires_grad = False

# Remplacer la dernière couche de classification
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 classes pour la sortie

# Supposons que train_images et train_labels soient vos données et étiquettes
# train_images = ...
# train_labels = ...

# Diviser les données en ensembles d'entraînement et de validation
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.2, random_state=42)

# Transformations

train_images = torch.tensor(train_images, dtype=torch.float32)
train_labels = torch.tensor(train_labels, dtype=torch.long)

val_images = torch.tensor(val_images, dtype=torch.float32)
val_labels = torch.tensor(val_labels, dtype=torch.long)

# Appliquer les transformations et créer des DataLoader
train_data = TensorDataset(train_images, train_labels)
val_data = TensorDataset(val_images, val_labels)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

# Critère de perte et optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Nombre d'époques pour l'entraînement
num_epochs = 10
liste_epoch  =[]
liste_train_loss = []
liste_val_loss = []
# Boucle d'entraînement
for epoch in range(num_epochs):
    model.train()  # Mode d'entraînement
    train_loss = 0.0

    for images, labels in train_loader:
        optimizer.zero_grad()  # Réinitialiser les gradients
        outputs = model(images)  # Passer les images dans le modèle
        loss = criterion(outputs, labels)  # Calculer la perte
        loss.backward()  # Rétropropagation
        optimizer.step()  # Mise à jour des poids
        train_loss += loss.item() * images.size(0)

    # Évaluation
    model.eval()  # Mode d'évaluation
    val_loss = 0.0

    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)

    # Calculer la perte moyenne par époque
    train_loss = train_loss / len(train_loader.dataset)
    val_loss = val_loss / len(val_loader.dataset)
    if epoch % 10 == 0:
        print(f'Époque {epoch+1}, Perte d\'entraînement: {train_loss:.4f}, Perte de validation: {val_loss:.4f}')
    liste_epoch.append(liste_epoch)
    liste_train_loss.append(liste_train_loss)
    liste_val_loss.append(liste_val_loss)
# Le modèle est maintenant entraîné et peut être utilisé pour des prédictions ou sauvegardé


Époque 1, Perte d'entraînement: 0.5653, Perte de validation: 0.3633
Époque 11, Perte d'entraînement: 0.0818, Perte de validation: 0.1085
Époque 21, Perte d'entraînement: 0.0595, Perte de validation: 0.0778


In [9]:
test_images = torch.tensor(test_images, dtype=torch.float32)
test_labels = torch.tensor(test_labels, dtype=torch.long)

test_dataset = TensorDataset(test_images, test_labels)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [10]:
# Calculer la précision sur les données de test
correct = 0
total = 0

model.eval()  # Mettre le modèle en mode d'évaluation

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Précision sur les données de test: {accuracy:.2f}%')

Précision sur les données de test: 97.50%


In [11]:
# Création du graphique
plt.figure(figsize=(10, 6))
plt.plot(liste_epoch, liste_train_loss, label='Perte d\'entraînement')
plt.plot(liste_epoch, liste_val_loss, label='Perte de validation', linestyle='--')

plt.xlabel('Époques')
plt.ylabel('Perte')
plt.title('Perte d\'entraînement et de validation au fil des époques')
plt.legend()
plt.grid(True)
plt.show()